# Import and Helper functions

In [5]:
# -- fix path --
from pathlib import Path
import sys
sys.path.append(str(Path('..').resolve()))
from source.resources import *
from source.metrics import *
from source.helper import *
from source.preprocessor import *
from source.constants import *
import Levenshtein
import wordfreq

from nltk import word_tokenize
from functools import lru_cache
import unicodedata
from collections import Counter
# from nltk import word_tokenize
import nltk
import magic
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.corpus import stopwords
# import spacy
# nlp = spacy.load("en_core_web_sm")

# from joblib import Memory
current_dir = Path('.')

DATASETS = [Dataset.BenchLS, Dataset.LexMTurk, Dataset.NNSeval, Dataset.SemEval2012, Dataset.TSAR_EN, Dataset.TSAR_ES, Dataset.TSAR_PT]


In [4]:
# def __normalize_string(text):
#         text = text.replace('``', '"')
#         text = text.replace('`', "'")
#         text = text.replace("''", '"')
#         text = text.strip('"')

#         return text 
    
# def sort_candidates_by_frequency(candidates, return_frequency=False):
#     '''
#     Default with frequency: [('wandered', 29), ('drifted', 4), ('walked', 4), ('traveled', 3), ('moved', 2), ('strolled', 2), ...]
    
#     Return candidats only: ['wandered', 'drifted', 'walked', 'traveled', 'moved', ...]
#     '''
#     candidates = Counter(candidates).items()
#     # [('pine', 16), ('tree', 16), ('evergreen', 10), ('pine tree', 2), ('cedar', 1)]
#     # [('important', 4), ('noticeable', 4), ('great', 3), ('known', 3), ('major', 3)]
#     candidates = sorted(candidates, key=lambda x: x[0]) # sort by alphabet when the frequencies are equal
#     candidates = sorted(candidates, key=lambda x: x[1], reverse=True) # sort by frequency
#     if not return_frequency:
#         candidates = [item[0] for item in candidates]
        
#     return candidates  
   
# def sort_candidates_by_ranking(candidates, return_ranking=False):
#     candidates = sorted(candidates, key=lambda x: x[0]) # sort by alphabet when the frequencies are equal
#     candidates = sorted(candidates, key=lambda x: x[1]) # sort by frequency
#     if not return_ranking:
#         candidates = [item[0] for item in candidates]
        
#     return candidates 
    
# def __load_data_with_index(dataset, return_frequency=False):
#         dataset_filepath = get_dataset_filepath(dataset)
#         lines = yield_lines(dataset_filepath)
#         docs = []

#         for line in lines:
#             chunks = line.split('\t')
#             text = chunks[0]
#             text = __normalize_string(text)
#             candidates = chunks[3:]
#             candidates = [tuple(candidate.split(':')[::-1]) for candidate in candidates]
            
#             # sort by alphabet and ranking
#             candidates = sort_candidates_by_ranking(candidates, return_ranking=return_frequency)
            
#             doc = {'text': text,
#                         'complex_word': chunks[1],
#                         'complex_word_index': chunks[2],
#                         'candidates': candidates}
#             docs.append(doc)
#         return pd.DataFrame(docs)
    

# def __load_data(dataset, return_frequency=False):
#     dataset_filepath = get_dataset_filepath(dataset)
#     lines = yield_lines(dataset_filepath)
#     docs = []

#     next(lines)  # skip the header
#     for line in lines:
#         chunks = line.split('\t')
#         text = chunks[0]
#         text = __normalize_string(text)
#         candidates = chunks[2:]
#         candidates = sort_candidates_by_frequency(candidates, return_frequency=return_frequency)
#         doc = {'text': text,
#                     'complex_word': chunks[1],
#                     'complex_word_index': word_index_in_text(text, chunks[1]),
#                     'candidates': candidates}
#         docs.append(doc)
#     return pd.DataFrame(docs)

# def load_data(dataset, return_frequency=False):
#     if dataset in [Dataset.LexMTurk, Dataset.TSAR_EN]:
#         data = __load_data(dataset, return_frequency=return_frequency)
#     else:
#         data = __load_data_with_index(dataset, return_frequency=return_frequency)
#     return data 



In [6]:
# line = "European Union foreign ministers agreed Monday to impose fresh sanctions on Syria as a U.N.-backed peace plan -- along with all other diplomatic efforts -- has yet to stop the carnage that mounts every day.	carnage	destruction	destruction	destruction	destruction	destruction	destruction	bloodshed	bloodshed	bloodshed	massacre	massacre	massacre	slaughter	slaughter	slaughter	carnage	carnage	brutality	butchering	butchery	damage	death	slaying	violence	war"
# chunks = line.split('\t')
# sentence = chunks[0]
# complex_word = chunks[1]
# candidates = chunks[2:]
# print(len(candidates))
# candidates = sort_candidates_by_frequency(candidates, return_frequency=True)
# # candidates = [f'{word}:{freq}' for word, freq in candidates]
# for word, freq in candidates:
#     print(f'{word}:{freq}', end=', ')

## Preview Data

### BenchLS

In [6]:
#BenchLS
data = load_dataset(Dataset.BenchLS)
data.head()

,text,complex_word,complex_word_index,candidates
0,"in march 1992 , linux version 0.95 was the fir...",pieces,35,"[[""parts""], [""bits""], [""components""], [""inform..."
1,much of the water carried by these streams is ...,diverted,9,"[[""redirected""], [""rerouted""], [""changed"", ""mo..."
2,harry also becomes the worthy possessor of the...,possessor,5,"[[""owner""], [""holder""], [""keeper""], [""buyer"", ..."
3,"escapologists escape from handcuffs , straitja...",perils,24,"[[""dangers""], [""hazards"", ""situations"", ""dange..."
4,"the storm continued , crossing the outer banks...",retained,13,"[[""kept""], [""held""], [""maintained""], [""regaine..."


### LexMTurk

In [9]:
data = load_dataset(Dataset.LexMTurk)
data

,text,complex_word,complex_word_index,candidates
0,"in march 1992 , linux version 0.95 was the fir...",pieces,34,"[[""parts""], [""bits""], [""components""], [""compon..."
1,much of the water carried by these streams is ...,diverted,9,"[[""redirected""], [""rerouted""], [""changed"", ""mo..."
2,harry also becomes the worthy possessor of the...,possessor,5,"[[""owner""], [""holder""], [""keeper""], [""buyer"", ..."
3,"escapologists escape from handcuffs , straitja...",perils,24,"[[""dangers""], [""difficulties"", ""danger"", ""rest..."
4,"the storm continued , crossing the outer banks...",retained,13,"[[""kept""], [""held""], [""maintained""], [""regaine..."
...,...,...,...,...
495,the marriage was marked from the outset by sex...,outset,6,"[[""beginning""], [""start""], [""begining""], [""beg..."
496,"born as alfons karg , he was trained as a tele...",profession,19,"[[""job""], [""career""], [""work"", ""job."", ""specia..."
497,"some features , however , are reminiscent of t...",reminiscent,6,"[[""similar""], [""remindful""], [""reminders""], [""..."
498,the shinto pantheon alone consists of an uncou...,consists,4,"[[""is made""], [""is made up""], [""is""], [""contai..."


### NNSeval

In [10]:
data = load_dataset(Dataset.NNSeval)
data

,text,complex_word,complex_word_index,candidates
0,harry also becomes the worthy possessor of the...,possessor,5,"[[""owner""], [""holder""], [""buyer"", ""master"", ""t..."
1,"escapologists escape from handcuffs , straitja...",perils,24,"[[""dangers""], [""danger"", ""situations"", ""hazard..."
2,"the storm continued , crossing the outer banks...",retained,13,"[[""kept""], [""regained""]]"
3,the united states convened a 13-nation confere...,convened,3,"[[""assembled""], [""gathered""], [""started""], [""o..."
4,dodd simply retained his athletic director pos...,acquired,11,"[[""received""], [""obtained""], [""achieved""], [""t..."
...,...,...,...,...
234,after adding the separate partnerships of jlw ...,worldwide,27,"[[""global""], [""international""]]"
235,"osis also provides its users with direct , yet...",worldwide,20,"[[""global""], [""international""]]"
236,"like maypoles , wreaths too , as she points ou...",worldwide,41,"[[""global""], [""international""]]"
237,these functions also use scheduling priority t...,functions,1,"[[""activities"", ""tasks""], [""operations"", ""sequ..."


### TSAR EN

In [3]:
data_tsar_en = load_dataset(Dataset.TSAR_EN)
data_tsar_en

,text,complex_word,complex_word_index,candidates
0,this discovery helped to establish yet another...,prototype,28,"[[""model"", ""sample""], [""prototype""], [""origina..."
1,putin was expected to formally register later ...,authoritarian,29,"[[""dictatorial""], [""controlling"", ""authoritari..."
2,"#7-8 despite the fog, other flights are report...",collision,18,"[[""crash""], [""impact"", ""accident""], [""collisio..."
3,"the larva grows to about 120-130 mm , and pupa...",pupates,9,"[[""grows""], [""pupates""], [""develops""], [""matur..."
4,gorbachev told the interfax news agency that a...,discontent,20,"[[""dissatisfaction""], [""unhappiness""], [""unres..."
...,...,...,...,...
381,"pearl s. buck died of lung cancer on march 6 ,...",interred,20,"[[""buried""], [""burried""], [""entombed"", ""inhume..."
382,his satirical street art and subversive epigra...,epigrams,6,"[[""quips"", ""jokes""], [""gems"", ""puns""], [""jest""..."
383,brevard county was the scene of six homicides ...,homicides,7,"[[""murders""], [""killings""], [""assassinations""]..."
384,#5-4 northern mali was seized by islamic milit...,seized,5,"[[""captured"", ""grabbed""], [""taken""], [""overtak..."


### TSAR ES

In [4]:
data_tsar_es = load_dataset(Dataset.TSAR_ES)
data_tsar_es

,text,complex_word,complex_word_index,candidates
0,"), fue un pianista y compositor folclórico, y ...",folclórico,7,"[[""tradicional""], [""pintoresco"", ""t\u00edpico""..."
1,"a comienzos de la década de 1980, se trasladó ...",labrarse,20,"[[""formarse""], [""construirse"", ""hacerse""], [""g..."
2,a lo largo de sus más de veinte años de experi...,tertulias,33,"[[""charlas""], [""reuniones""], [""debates""], [""co..."
3,a pesar de las pocas bajas (menos de 500 en to...,propiciado,38,"[[""provocado""], [""favorecido"", ""desencadenado""..."
4,a pesar de las pocas bajas (menos de 500 en to...,decadencia,40,"[[""ca\u00edda""], [""debilitaci\u00f3n""], [""caid..."
...,...,...,...,...
376,una bomba peristáltica es un tipo de bomba hid...,hidráulica,8,"[[""hidr\u00e1ulica""], [""hidromec\u00e1nica""], ..."
377,"una noche,un extraño personaje llamado dedo po...",secuaces,53,"[[""seguidores""], [""c\u00f3mplices""], [""compa\u..."
378,"utilizó el seudónimo ""etpa"" (ermelinda talea p...",seudónimo,2,"[[""alias""], [""apodo"", ""sobrenombre""], [""nombre..."
379,"villasequilla se llamó antes ""villaseca de val...",dependencia,18,"[[""vinculaci\u00f3n""], [""subordinaci\u00f3n""],..."


### TSAR PT

In [5]:
data_tsar_pt = load_dataset(Dataset.TSAR_PT)
data_tsar_pt

,text,complex_word,complex_word_index,candidates
0,ao prever a distribuição gratuita de uniforme ...,controvérsia,20,"[[""pol\u00eamica""], [""contradi\u00e7\u00e3o""],..."
1,a matéria ainda precisa passar pela câmara dos...,matéria,1,"[[""assunto"", ""lei""], [""proposta"", ""conte\u00fa..."
2,o acessório polêmico entrou no projeto de auto...,polêmico,2,"[[""controverso""], [""discut\u00edvel""], [""duvid..."
3,toda a operação teria sido armada para vingar ...,armada,5,"[[""criada""], [""planejada""], [""organizada"", ""pr..."
4,blitze barreiras e operações em profusão foram...,profusão,5,"[[""abund\u00e2ncia""], [""fartura"", ""quantidade""..."
...,...,...,...,...
381,para provar a idéia os pesquisadores compilara...,compilaram,6,"[[""reuniram""], [""juntaram""], [""selecionaram""],..."
382,o estudo estará numa edição futura do periódic...,científico,8,"[[""tecnol\u00f3gico"", ""t\u00e9cnico""], [""mestr..."
383,cremos que elas seriam mais eficientes que os ...,inibir,11,"[[""impedir""], [""diminuir""], [""impossibilitar""]..."
384,a ficção científica americana na forma de livr...,fascinada,12,"[[""encantada"", ""apaixonada""], [""deslumbrado""],..."


# Check frequency

In [40]:
data = load_data(Dataset.LexMTurk, return_frequency=True)

items = []
for row in data.iloc:
    candidates = row['candidates']
    if len(candidates) > 1 and candidates[0][1] == candidates[1][1]:
        print(candidates[:5])
        items.append(items)
print(Dataset.LexMTurk, len(data), len(items)) 


[('assembled', 11), ('called', 11), ('gathered', 6), ('held', 5), ('met', 3)]
[('pine', 16), ('tree', 16), ('evergreen', 10), ('pine tree', 2), ('cedar', 1)]
[('important', 4), ('noticeable', 4), ('great', 3), ('known', 3), ('major', 3)]
[('almost never', 6), ('not often', 6), ('hardly', 5), ('hardly ever', 5), ('not usually', 5)]
[('registered', 8), ('signed up', 8), ('studied', 7), ('started', 6), ('joined', 5)]
[('past', 16), ('previous', 16), ('old', 8), ('prior', 4), ('earlier', 2)]
[('ideas', 10), ('rules', 10), ('beliefs', 7), ('laws', 5), ('principles', 2)]
[('honest', 16), ('reliable', 16), ('believable', 5), ('credible', 3), ('dependable', 3)]
[('concentrated', 8), ('interested', 8), ('centered', 6), ('fixed', 5), ('fixated', 3)]
[('before', 13), ('earlier', 13), ('already', 5), ('in the past', 4), ('once', 4)]
[('decides', 11), ('determines', 11), ('shows', 5), ('tells', 4), ('causes', 3)]
[('mated', 5), ('raised', 5), ('made', 4), ('added', 3), ('connected', 3)]
[('ones', 8

In [41]:

data = load_data(Dataset.NNSeval, return_frequency=True)

items = []
for row in data.iloc:
    candidates = row['candidates']
    if len(candidates) > 1 and candidates[0][1] == candidates[1][1]:
        print(candidates[:5])
        items.append(items)
print(Dataset.NNSeval, len(data), len(items)) 

[('blockade', '1'), ('capture', '1'), ('battle', '2'), ('invasion', '3'), ('surrounding', '3')]
[('dream', '1'), ('try', '1'), ('aim', '2'), ('strive', '2'), ('want', '2')]
[('born', '1'), ('established', '1'), ('founded', '1'), ('started', '1')]
[('named', '1'), ('selected', '1'), ('elected', '2'), ('chosen as', '3'), ('designated', '3')]
[('entirely', '1'), ('soley', '1'), ('singularly', '2')]
[('old', '1'), ('present-day', '1'), ('historical', '2'), ('mainstream', '2'), ('popular', '2')]
[('important', '1'), ('noticeable', '1'), ('great', '2'), ('known', '2'), ('major', '2')]
[('earned', '1'), ('obtained', '1'), ('received', '1'), ('aquired', '2'), ('gave', '2')]
[('eventually', '1'), ('increasingly', '1'), ('gently', '2'), ('little by little', '2'), ('over time', '2')]
[('daid', '1'), ('sayed', '1')]
[('cape', '1'), ('point', '1'), ('area', '2'), ('tip', '2'), ('landmass', '3')]
[('developing', '1'), ('emerging', '1'), ('budding', '2'), ('abandoned', '3'), ('blossoming', '3')]
[('a

In [42]:

data = load_data(Dataset.BenchLS, return_frequency=True)

items = []
for row in data.iloc:
    candidates = row['candidates']
    if len(candidates) > 1 and candidates[0][1] == candidates[1][1]:
        print(candidates[:5])
        items.append(items)
print(Dataset.BenchLS, len(data), len(items)) 

[('assembled', '1'), ('called', '1'), ('gathered', '2'), ('held', '3'), ('met', '4')]
[('pine', '1'), ('tree', '1'), ('evergreen', '2'), ('cedar', '3'), ('fir', '3')]
[('important', '1'), ('noticeable', '1'), ('great', '2'), ('known', '2'), ('major', '2')]
[('hardly', '1'), ('seldom', '1'), ('sometimes', '1'), ('not', '2'), ('barely', '3')]
[('past', '1'), ('previous', '1'), ('old', '2'), ('prior', '3'), ('earlier', '4')]
[('ideas', '1'), ('rules', '1'), ('beliefs', '2'), ('laws', '3'), ('principles', '4')]
[('honest', '1'), ('reliable', '1'), ('believable', '2'), ('credible', '3'), ('dependable', '3')]
[('concentrated', '1'), ('interested', '1'), ('centered', '2'), ('fixed', '3'), ('fixated', '4')]
[('decides', '1'), ('determines', '1'), ('shows', '2'), ('tells', '3'), ('causes', '4')]
[('mated', '1'), ('raised', '1'), ('made', '2'), ('added', '3'), ('connected', '3')]
[('biggest', '1'), ('main', '1'), ('first', '2'), ('greatest', '2'), ('best', '3')]
[('ones', '1'), ('types', '1'), (

# Database similarity check


In [43]:
# memory = Memory(current_dir / 'cache', verbose=False)

# @memory.cache()
def sentence_similarity(sentence, sentences):
    scores = []
    for s in sentences:
        score = Levenshtein.ratio(sentence, s)
        scores.append((s, score))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[0]

def compare_two_datasets(dataset1, dataset2):
    # use Levenshtein similarity to check
    data1 = load_data(dataset1)
    data2 = load_data(dataset2)
    # benchls
    sentences = [row['text'].lower() for row in data2.iloc]
    similarity_scores = []
    for i in tqdm(range(len(data1))):
        row = data1.iloc[i]
        sent = row['text'].lower()
        scores = sentence_similarity(sent, sentences)
        similarity_scores.append((sent, scores))    

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1][1], reverse=True)
    nb_similar_sent = 0
    for sent, scores in similarity_scores:
        sent2, score = scores
        if score > 0.9:
            nb_similar_sent +=1
        if score > 0.5:
            print(f'\n{sent}')
            print(sent2, score)
    print(f'Compare {dataset1} and {dataset2}')
    print(f'Total Count of similar sentences: {nb_similar_sent}')


## BenchLS and LexMTurk

In [44]:
compare_two_datasets(Dataset.BenchLS, Dataset.LexMTurk)

100%|██████████| 929/929 [00:02<00:00, 329.47it/s]


much of the water carried by these streams is diverted .
much of the water carried by these streams is diverted . 1.0

harry also becomes the worthy possessor of the remaining deathly hallows : the invisibility cloak and the resurrection stone , hence becoming the true master of death .
harry also becomes the worthy possessor of the remaining deathly hallows : the invisibility cloak and the resurrection stone , hence becoming the true master of death . 1.0

escapologists escape from handcuffs , straitjackets , cages , coffins , steel boxes , barrels , bags , burning buildings , fish-tanks and other perils , often in combination .
escapologists escape from handcuffs , straitjackets , cages , coffins , steel boxes , barrels , bags , burning buildings , fish-tanks and other perils , often in combination . 1.0

the storm continued , crossing the outer banks of north carolina , and retained its strength until june 20 when it became extratropical near newfoundland .
the storm continued , cr

## BenchLS and NNSeval

In [45]:
compare_two_datasets(Dataset.BenchLS, Dataset.NNSeval)

100%|██████████| 929/929 [00:01<00:00, 565.03it/s]


harry also becomes the worthy possessor of the remaining deathly hallows : the invisibility cloak and the resurrection stone , hence becoming the true master of death .
harry also becomes the worthy possessor of the remaining deathly hallows : the invisibility cloak and the resurrection stone , hence becoming the true master of death . 1.0

escapologists escape from handcuffs , straitjackets , cages , coffins , steel boxes , barrels , bags , burning buildings , fish-tanks and other perils , often in combination .
escapologists escape from handcuffs , straitjackets , cages , coffins , steel boxes , barrels , bags , burning buildings , fish-tanks and other perils , often in combination . 1.0

the storm continued , crossing the outer banks of north carolina , and retained its strength until june 20 when it became extratropical near newfoundland .
the storm continued , crossing the outer banks of north carolina , and retained its strength until june 20 when it became extratropical near ne

## BenchLS and TSAR_EN

In [2]:
compare_two_datasets(Dataset.BenchLS, Dataset.TSAR_EN)

NameError: name 'compare_two_datasets' is not defined

## BenchLS and SemEval2012

In [47]:
compare_two_datasets(Dataset.BenchLS, Dataset.SemEval2012)

100%|██████████| 929/929 [00:09<00:00, 101.50it/s]


suddenly there was a scattering of fire , which three outfielders caught the brunt ; the centerfield was hit and was captured , left and right field managed to get back to our lines .
suddenly there was a scattering of fire , which three outfielders caught the brunt ; the centerfield was hit and was captured , left and right field managed to get back to our lines . 1.0

 c158 ( cf ex r145 ) " the provincial in councilor the superior ofthe delegation in councilsets the financial competency of local superiorsand their councilsand determines which assets can be managed by individualoblates and by local superiors and their councils .
 c158 ( cf ex r145 ) " the provincial in councilor the superior ofthe delegation in councilsets the financial competency of local superiorsand their councilsand determines which assets can be managed by individualoblates and by local superiors and their councils . 1.0

business development manager strategic alliances this individual will be responsible for co

In [48]:
compare_two_datasets(Dataset.NNSeval, Dataset.SemEval2012)

100%|██████████| 239/239 [00:02<00:00, 103.72it/s]


business development manager strategic alliances this individual will be responsible for coordinating , negotiating , launching , and managing accuity 's partner relationships .
business development manager strategic alliances this individual will be responsible for coordinating , negotiating , launching , and managing accuity 's partner relationships . 1.0

it is relatively simple to find lenses which will maintain reasonably good central resolution the central resolution , but thus far , i have not been able to achieve an edge resolution better than about 0.1 mm with " off the shelf " lenses .
it is relatively simple to find lenses which will maintain reasonably good central resolution the central resolution , but thus far , i have not been able to achieve an edge resolution better than about 0.1 mm with " off the shelf " lenses . 1.0

the calculation is based on the loan 's original principal , so fellows who are not recent graduates , and thus have been making loan payments , are 

## TSAR EN

In [49]:
compare_two_datasets(Dataset.TSAR_EN, Dataset.BenchLS)

100%|██████████| 385/385 [00:01<00:00, 201.09it/s]


hundreds were beaten and detained.
hurricane-force wind gusts were reported in new england . 0.5494505494505495

we do feel we are prevailing in this with our afghan partners, he said.
but he could feel the whale was sensing him with sound pulses . 0.5373134328358209

we do feel we are prevailing in this with our afghan partners," he said.
but he could feel the whale was sensing him with sound pulses . 0.5333333333333333

that prompted the military to deploy its largest warship, the brp gregorio del pilar, which was recently acquired from the united states.
the majority of subjects , 90 percent , came from the metropolitan seattle area , with the remainder coming from across the united states . 0.5289855072463768

the official spoke on condition of anonymity because he was not authorized to speak to the press.
the geological composition of rock and soil dictates the type of shore which is created . 0.5268817204301075

the official spoke on condition of anonymity because he was not aut

In [50]:
compare_two_datasets(Dataset.TSAR_EN, Dataset.NNSeval)

100%|██████████| 385/385 [00:00<00:00, 565.34it/s]


#5-4 northern mali was seized by islamic militants in 2012, with france leading a military intervention the following year.
the city was first founded by the british in 1827 , who leased the island from spain during the colonial period . 0.5169491525423728

#7-8 despite the fog, other flights are reported to have landed safely leading up to the collision.
you should therefore report the break-in to the police . 0.5161290322580645

her older sister, aged 21, lived at the rented house, in a recently built development at the back of the established housing estate.
located on the river pedieos and situated almost in the center of the island , it is the seat of government as well as the main business center . 0.5126353790613718

#2-19 robert added the group knew travel and work in the area was dangerous, and took precautions.
after the partitions of poland in 1794 the area was annexed by prussia . 0.5058823529411764

the order was the first national decoration awarded by the japanese gover

In [51]:
compare_two_datasets(Dataset.TSAR_EN, Dataset.LexMTurk)

100%|██████████| 385/385 [00:01<00:00, 344.21it/s]


hundreds were beaten and detained.
hurricane-force wind gusts were reported in new england . 0.5494505494505495

the official spoke on condition of anonymity because he was not authorized to speak to the press.
the geological composition of rock and soil dictates the type of shore which is created . 0.5268817204301075

the official spoke on condition of anonymity because he was not authorized to speak to the press.
the geological composition of rock and soil dictates the type of shore which is created . 0.5268817204301075

the chinese embassy accused the philippine warship of harassing the fishermen and called for it to leave chinese territory.
the pennines constitute the main watershed in northern england , dividing the eastern and western parts of the country . 0.5185185185185185

the shootings are the worst homicides to take place in brevard county since 1987.
the convent has been the official residence of the governor of gibraltar since 1728 . 0.5180722891566265

#5-4 northern mal

# Features Analysis
---

## CharRatioFeature

In [52]:
def char_ratio_check(dataset, i_candidate=0, print_words=False):
    print(dataset, i_candidate)
    
    data = load_data(dataset)
    size = len(data)
    count_bigger = 0 
    count_smaller = 0
    equal = 0
    length_dict = {}
    for row in data.iloc:
        complex = row['complex_word']
        candidates = row['candidates']
        
        candidate = candidates[i_candidate] if len(candidates) > i_candidate else candidates[0]
        
        # print(complex, candidates)
        if print_words:
            print(f'{complex:15} {candidate:15} {len(complex):15} {len(candidate):15}')
        if len(complex) > len(candidate):
            count_bigger += 1
        elif len(complex) < len(candidate):
            count_smaller += 1
        else:
            equal += 1 
    print(f'Complex is longer than candidate: {count_bigger / size * 100 :.2f}%')
    print(f'Complex is shorter than candidate: {count_smaller / size * 100 :.2f}%')
    print(f'Complex is equal to candidate: {equal / size * 100 :.2f}%')



In [53]:
for dataset in DATASETS:
    char_ratio_check(dataset, i_candidate=0)
    char_ratio_check(dataset, i_candidate=1)
    char_ratio_check(dataset, i_candidate=2)
    char_ratio_check(dataset, i_candidate=3)
    char_ratio_check(dataset, i_candidate=4)
    print()

BenchLS 0
Complex is longer than candidate: 69.32%
Complex is shorter than candidate: 22.39%
Complex is equal to candidate: 8.29%
BenchLS 1
Complex is longer than candidate: 59.20%
Complex is shorter than candidate: 28.42%
Complex is equal to candidate: 12.38%
BenchLS 2
Complex is longer than candidate: 54.36%
Complex is shorter than candidate: 33.15%
Complex is equal to candidate: 12.49%
BenchLS 3
Complex is longer than candidate: 57.91%
Complex is shorter than candidate: 31.97%
Complex is equal to candidate: 10.12%
BenchLS 4
Complex is longer than candidate: 58.56%
Complex is shorter than candidate: 31.22%
Complex is equal to candidate: 10.23%

lex.mturk 0
Complex is longer than candidate: 77.96%
Complex is shorter than candidate: 14.43%
Complex is equal to candidate: 7.62%
lex.mturk 1
Complex is longer than candidate: 66.93%
Complex is shorter than candidate: 20.24%
Complex is equal to candidate: 12.83%
lex.mturk 2
Complex is longer than candidate: 64.13%
Complex is shorter than can

## WordRank

In [54]:

def word_rank_check(dataset, i_candidate=0, verbose=False):
    data = load_data(dataset)
    word_rank = WordRank()
    count = 0
    for row in data.iloc:
        complex_word = row['complex_word']
        candidates = row['candidates']
        # candidate = candidates[0]
        candidate = candidates[i_candidate] if len(candidates) > i_candidate else candidates[0]
        # if candidate == complex_word and len(candidates) > 1: candidate = candidates[1]
        
        score_complex = word_rank.get_lexical_complexity_score(complex_word)
        score_candidate = word_rank.get_lexical_complexity_score(candidate)
        ratio = score_candidate / score_complex
        if ratio < 1: count += 1
        # if ratio == 1: print(complex_word, candidate)
        if verbose:
            print(f'{complex_word:10} {candidate:15} {score_complex:.2f} {score_candidate:.2f} {ratio:.2f}') 
        
    print(f'{dataset} {i_candidate:15} : {count / len(data) * 100:.2f}')
    # print()

In [55]:
for dataset in DATASETS:
    word_rank_check(dataset)
    word_rank_check(dataset, i_candidate=1)
    word_rank_check(dataset, i_candidate=2)
    word_rank_check(dataset, i_candidate=3)
    word_rank_check(dataset, i_candidate=4)
    
# SemEval 2012 includes many first candidates the same as the complex words.

BenchLS               0 : 84.61
BenchLS               1 : 70.40
BenchLS               2 : 61.46
BenchLS               3 : 62.11
BenchLS               4 : 67.06
lex.mturk               0 : 91.18
lex.mturk               1 : 81.36
lex.mturk               2 : 78.96
lex.mturk               3 : 75.95
lex.mturk               4 : 72.75
NNSeval               0 : 75.31
NNSeval               1 : 68.20
NNSeval               2 : 61.09
NNSeval               3 : 65.69
NNSeval               4 : 66.11
semeval2012               0 : 55.22
semeval2012               1 : 46.52
semeval2012               2 : 38.51
semeval2012               3 : 33.68
semeval2012               4 : 34.23
tsar-en               0 : 85.45
tsar-en               1 : 74.29
tsar-en               2 : 69.09
tsar-en               3 : 66.75
tsar-en               4 : 64.42


## WordSyllableRatioFeature

In [56]:

def word_syllables_check(dataset, verbose=False):
    data = load_data(dataset)
    feature = WordSyllable()
    count = 0
    for row in data.iloc:
        complex_word = row['complex_word']
        candidates = row['candidates']
        candidate = candidates[0]
        # if candidate == complex_word and len(candidates) > 1: candidate = candidates[1]
        
        nb_syllable_complex = feature.count_syllable(complex_word)
        nb_syllable_candidate = feature.count_syllable(candidate)
        ratio = safe_division(nb_syllable_candidate, nb_syllable_complex)
        # ratio = safe_division(nb_syllable_complex, nb_syllable_candidate)
        
        if ratio < 1: count += 1
        # if ratio == 1: print(complex_word, candidate)
        
        if verbose:
            print(f'{complex_word:10} {candidate:15} {nb_syllable_complex:.2f} {nb_syllable_candidate:.2f} {ratio:.2f}') 
        
    print(f'{dataset:12}: {count / len(data) * 100:.2f}')
    print()

In [57]:
for dataset in DATASETS:
    word_syllables_check(dataset)

BenchLS     : 59.85

lex.mturk   : 67.33

NNSeval     : 48.54

semeval2012 : 30.25

tsar-en     : 50.65



## CandidateRankingRatioFeature

In [58]:

def candidate_ranking_check(dataset, verbose=False):
    data = load_data(dataset)
    feature = CandidateRanking()
    count = 0
    for row in data.iloc:
        complex_word = row['complex_word']
        candidates = row['candidates']
        candidate = candidates[0]
        # if candidate == complex_word and len(candidates) > 1: candidate = candidates[1]
        
        # score_complex = feature.count_syllable(complex_word)
        # score_candidate = feature.count_syllable(candidate)
        # ratio = safe_division(score_candidate, score_complex)
        ratio = feature.get_ranking_ratio()
        
        if ratio < 1: count += 1
        # if ratio == 1: print(complex_word, candidate)
        
        if verbose:
            print(f'{complex_word:10} {candidate:15} {score_complex:.2f} {score_candidate:.2f} {ratio:.2f}') 
        
    print(f'{dataset:12}: {count / len(data) * 100:.2f}')
    print()

# Cosine Similarity - BERT

In [80]:
import json 
from minicons import cwe # https://github.com/kanishkamisra/minicons
from sklearn.metrics.pairwise import cosine_similarity
from functools import lru_cache
from diskcache import Cache
cache = Cache(CACHE_DIR)

@lru_cache(maxsize=10)
def get_model(lang='en'):
    if lang == 'en':
        return cwe.CWE('bert-base-uncased')
    elif lang == 'es':
        return cwe.CWE('dccuchile/bert-base-spanish-wwm-uncased')
    elif lang == 'pt':
        return cwe.CWE('')
    elif lang == 'de':
        pass 
    elif lang == 'fr':
        pass 

# cache.evict(tag='get_cosine_similarity')

@cache.memoize(tag='get_cosine_similarity')
def get_cosine_similarity(text, complex_word, candidate):
    context_words = [(text, complex_word), (text.replace(complex_word, candidate), candidate)]
    model = get_model()
    embeddings = model.extract_representation(context_words, layer=12)
    # embeddings = model.extract_representation(context_words, layer=[-1])
    
    return cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))[0][0]
    
data = load_dataset(Dataset.TSAR_EN)

In [81]:
# for dataset in DATASETS:
#     word_syllables_check(dataset)

row = data.iloc[2]
text = row['text']
complex_word = row['complex_word']
candidates = row['candidates']
print(text)
print(complex_word, candidates)

candidates = json.loads(row['candidates'])
candidates = [candidate for items in candidates for candidate in items]
print(candidates)
for candidate in candidates: 
    print(f"{row['complex_word']} {candidate:<15} {get_cosine_similarity(row['text'], row['complex_word'], candidate)}")


#7-8 despite the fog, other flights are reported to have landed safely leading up to the collision.
collision [["crash"], ["impact", "accident"], ["collision"], ["clash", "striking"]]
['crash', 'impact', 'accident', 'collision', 'clash', 'striking']
collision crash           0.8515915274620056
collision impact          0.8278858661651611
collision accident        0.8578176498413086
collision collision       0.9999998807907104
collision clash           0.6809971928596497
collision striking        0.6488008499145508


In [82]:
for i in range(len(data[:10])):
    row = data.iloc[i]
    text = row['text']
    complex_word = row['complex_word']
    candidates = json.loads(row['candidates'])
    candidates = [candidate for items in candidates for candidate in items]
    # print('='*80)
    # print(text)
    # print(complex_word)
    # print(candidates)
    candidate = candidates[0]
    print(f"{row['complex_word']:<15} {candidate:<15} {get_cosine_similarity(row['text'], row['complex_word'], candidate)}")
    
    

prototype       model           0.7478899955749512
authoritarian   dictatorial     0.8635026812553406
collision       crash           0.8515915274620056
pupates         grows           0.7571403384208679
discontent      dissatisfaction 0.8958964347839355
decomposes      decays          0.8171378970146179
descent         landing         0.7276878356933594
sectarian       divided         0.8202275037765503
sectarian       sectarian       0.9999998807907104
offshoot        branch          0.8018960952758789


In [83]:
for i in range(len(data[:10])):
    row = data.iloc[i]
    text = row['text']
    complex_word = row['complex_word']
    candidates = json.loads(row['candidates'])
    candidates = [candidate for items in candidates for candidate in items]
    candidates = remove_word_from_list(complex_word, candidates)
    candidate = candidates[0]
    score1 = get_cosine_similarity(row['text'], row['complex_word'], candidate)
    candidate2 = candidates[1]
    score2 = get_cosine_similarity(row['text'], row['complex_word'], candidate2)
    print(f"{row['complex_word']:<15} {candidate:<15} {score1:.2f} {candidate2:<15} {score2:.2f}")
    
    

prototype       model           0.75 sample          0.66
authoritarian   dictatorial     0.86 controlling     0.79
collision       crash           0.85 impact          0.83
pupates         grows           0.76 develops        0.81
discontent      dissatisfaction 0.90 unhappiness     0.75
decomposes      decays          0.82 breaks down     0.77
descent         landing         0.73 drop            0.81
sectarian       divided         0.82 denoting        0.63
sectarian       religious       0.76 dissenter       0.87
offshoot        branch          0.80 subsidiary      0.78


In [84]:
def process_candidates(complex_word, candidates_str):
    # convert from string to array, flatten the array from 2d to 1d, and remove the candidate which is the same as the complex word
    candidates = json.loads(candidates_str)
    candidates = [candidate for items in candidates for candidate in items]
    candidates = remove_word_from_list(complex_word, candidates) # remove candidate == complex word
    return candidates 

def get_candidates_cosine_similarity(text,complex_word, candidates):
    return {candidate: get_cosine_similarity(text, complex_word, candidate) for candidate in candidates}

def find_candidate_with_highest_similarity_score(similarity_scores):
    max_key = max(similarity_scores, key=similarity_scores.get)
    return max_key, similarity_scores[max_key]
    
def cosine_similarity_check(i_candidate=0):
    
    size = len(data)
    # size = 10
    count = 0
    for i in tqdm(range(size)):
        row = data.iloc[i]
        text = row['text']
        complex_word = row['complex_word']
        candidates = process_candidates(complex_word, row['candidates'])
        similarity_scores = get_candidates_cosine_similarity(text, complex_word, candidates)
        max_candidate, max_score = find_candidate_with_highest_similarity_score(similarity_scores)
        if i_candidate < len(candidates):
            candidate = candidates[i_candidate]
            score = similarity_scores[candidate]
            # print('='*80)
            # print(candidates)
            if score == max_score: count +=1
        # print(f'{complex_word:<15} {candidate:<15} {score:.2f} {max_candidate:<15} {max_score:.2f}')
    print(f'{i_candidate}: {count/size:.2f}')
    
for i in range(5):
    cosine_similarity_check(i)
    

100%|██████████| 386/386 [02:59<00:00,  2.15it/s]


0: 0.26


100%|██████████| 386/386 [00:00<00:00, 2924.51it/s]


1: 0.17


100%|██████████| 386/386 [00:00<00:00, 3126.49it/s]


2: 0.10


100%|██████████| 386/386 [00:00<00:00, 3091.26it/s]


3: 0.12


100%|██████████| 386/386 [00:00<00:00, 3180.80it/s]

4: 0.08


# Max Sequence Length

#

In [3]:

    
def get_length(text):
    return len(word_tokenize(text))

def get_min_max_seq_length(dataset):
    data = load_dataset(dataset)
    l = [get_length(line) for line in data['text']]
             
    return min(l), max(l)

def get_avg_seq_length(data):
    data = load_dataset(dataset)
    lengths = [get_length(line) for line in data['text']]
    return np.mean(lengths)
    
for dataset in DATASETS:
    min_length, max_length = get_min_max_seq_length(dataset)
    print(f'{dataset} \t {min_length} \t {max_length} \t {get_avg_seq_length(dataset):.2f}')
    

BenchLS 	 6 	 100 	 27.90
lex.mturk 	 6 	 78 	 26.23
NNSeval 	 7 	 78 	 27.95
semeval2012 	 5 	 92 	 28.23
tsar-en 	 6 	 83 	 29.85
tsar-es 	 5 	 138 	 35.14
tsar-pt 	 3 	 57 	 23.12


# Length distribution

In [62]:
# import plotly.graph_objects as go

# fig = go.Figure(go.Bar(
#             x=[20, 14, 23],
#             y=['giraffes', 'orangutans', 'monkeys'],
#             orientation='h'))
import plotly.graph_objects as go
def plot_length_distibution(dataset):
    # dataset = Dataset.TSAR_EN

    data = load_dataset(dataset)
    dict = {}
    for line in data['text']:
        l = str(get_length(line))
        if l in dict:
            dict[l] += 1
        else:
            dict[l] = 1
    vals = tuple((key, val) for key, val in dict.items())
    vals = sorted(vals, key=lambda x: x[1], reverse=True)
    x = [i[0] for i in vals]
    y = [i[1] for i in vals]
    fig = go.Figure(go.Bar(
                x=x,
                y=y,
                orientation='v'))
    fig.show()
    print('Dataset: ', dataset)
    print('Max length: ', max(v[0] for v in vals))
    print('(len, count) (sorted by count):', vals)
    print('(len, count) (sorted by len): ', sorted(vals, key=lambda x: int(x[0]), reverse=True))

In [63]:
for dataset in DATASETS:
    plot_length_distibution(dataset)
    

Dataset:  BenchLS
Max length:  9
(len, count) (sorted by count): [('24', 37), ('18', 35), ('25', 35), ('23', 35), ('20', 34), ('19', 34), ('14', 31), ('30', 30), ('17', 30), ('16', 30), ('28', 29), ('15', 29), ('31', 28), ('26', 27), ('21', 27), ('22', 27), ('27', 26), ('33', 25), ('34', 21), ('11', 20), ('32', 20), ('35', 19), ('13', 18), ('39', 17), ('37', 17), ('10', 17), ('12', 16), ('29', 15), ('42', 13), ('40', 12), ('36', 11), ('43', 10), ('49', 10), ('44', 9), ('45', 9), ('38', 9), ('9', 9), ('47', 8), ('41', 8), ('46', 8), ('48', 8), ('8', 6), ('58', 6), ('50', 6), ('7', 5), ('59', 4), ('55', 4), ('51', 4), ('56', 3), ('66', 3), ('62', 3), ('67', 3), ('52', 2), ('78', 2), ('54', 2), ('6', 2), ('57', 2), ('74', 2), ('61', 2), ('65', 2), ('79', 2), ('77', 1), ('73', 1), ('76', 1), ('64', 1), ('71', 1), ('60', 1), ('72', 1), ('100', 1), ('75', 1), ('53', 1), ('68', 1)]
(len, count) (sorted by len):  [('100', 1), ('79', 2), ('78', 2), ('77', 1), ('76', 1), ('75', 1), ('74', 2), ('

Dataset:  lex.mturk
Max length:  9
(len, count) (sorted by count): [('18', 24), ('30', 22), ('15', 22), ('24', 21), ('20', 21), ('25', 21), ('19', 21), ('23', 20), ('27', 19), ('17', 18), ('28', 17), ('22', 17), ('14', 17), ('16', 16), ('26', 15), ('31', 14), ('21', 13), ('13', 11), ('33', 11), ('29', 10), ('11', 9), ('12', 9), ('37', 9), ('10', 9), ('32', 9), ('39', 9), ('35', 7), ('36', 7), ('34', 7), ('38', 4), ('8', 4), ('44', 4), ('45', 4), ('47', 4), ('41', 4), ('40', 4), ('43', 4), ('42', 4), ('9', 4), ('49', 3), ('48', 3), ('58', 2), ('52', 2), ('54', 2), ('6', 2), ('46', 2), ('74', 2), ('60', 2), ('56', 1), ('78', 1), ('59', 1), ('50', 1), ('66', 1), ('57', 1), ('77', 1), ('73', 1), ('76', 1), ('55', 1), ('7', 1), ('64', 1), ('71', 1), ('51', 1), ('65', 1)]
(len, count) (sorted by len):  [('78', 1), ('77', 1), ('76', 1), ('74', 2), ('73', 1), ('71', 1), ('66', 1), ('65', 1), ('64', 1), ('60', 2), ('59', 1), ('58', 2), ('57', 1), ('56', 1), ('55', 1), ('54', 2), ('52', 2), ('51

Dataset:  NNSeval
Max length:  8
(len, count) (sorted by count): [('30', 13), ('25', 13), ('28', 12), ('24', 12), ('20', 12), ('15', 11), ('14', 10), ('26', 9), ('21', 9), ('22', 9), ('23', 8), ('31', 8), ('27', 7), ('12', 6), ('29', 6), ('10', 6), ('33', 6), ('35', 5), ('19', 5), ('42', 5), ('39', 5), ('17', 4), ('16', 4), ('43', 4), ('37', 3), ('18', 3), ('38', 3), ('32', 3), ('48', 3), ('45', 2), ('41', 2), ('8', 2), ('40', 2), ('11', 2), ('44', 2), ('46', 2), ('58', 2), ('34', 2), ('55', 2), ('49', 2), ('57', 1), ('36', 1), ('74', 1), ('65', 1), ('67', 1), ('13', 1), ('59', 1), ('50', 1), ('51', 1), ('78', 1), ('7', 1), ('66', 1), ('68', 1)]
(len, count) (sorted by len):  [('78', 1), ('74', 1), ('68', 1), ('67', 1), ('66', 1), ('65', 1), ('59', 1), ('58', 2), ('57', 1), ('55', 2), ('51', 1), ('50', 1), ('49', 2), ('48', 3), ('46', 2), ('45', 2), ('44', 2), ('43', 4), ('42', 5), ('41', 2), ('40', 2), ('39', 5), ('38', 3), ('37', 3), ('36', 1), ('35', 5), ('34', 2), ('33', 6), ('32',

Dataset:  semeval2012
Max length:  92
(len, count) (sorted by count): [('19', 77), ('17', 71), ('18', 70), ('20', 66), ('26', 66), ('32', 65), ('24', 64), ('25', 64), ('16', 63), ('21', 62), ('22', 60), ('29', 60), ('14', 59), ('15', 59), ('27', 55), ('30', 53), ('28', 53), ('23', 52), ('35', 51), ('33', 48), ('31', 47), ('11', 46), ('34', 42), ('13', 42), ('12', 39), ('36', 36), ('10', 33), ('39', 30), ('42', 28), ('40', 28), ('38', 25), ('37', 25), ('9', 24), ('46', 23), ('43', 23), ('41', 22), ('8', 20), ('49', 19), ('48', 19), ('44', 15), ('7', 15), ('45', 12), ('60', 12), ('51', 12), ('53', 12), ('58', 10), ('50', 9), ('47', 9), ('55', 9), ('62', 9), ('67', 7), ('56', 7), ('59', 7), ('52', 7), ('63', 6), ('54', 6), ('68', 5), ('57', 5), ('66', 4), ('6', 4), ('61', 4), ('79', 4), ('69', 3), ('70', 3), ('75', 3), ('71', 3), ('65', 3), ('76', 2), ('78', 2), ('72', 2), ('80', 2), ('92', 1), ('91', 1), ('77', 1), ('90', 1), ('64', 1), ('74', 1), ('5', 1), ('73', 1)]
(len, count) (sorte

Dataset:  tsar-en
Max length:  9
(len, count) (sorted by count): [('22', 23), ('29', 19), ('21', 16), ('30', 15), ('28', 15), ('37', 15), ('35', 15), ('20', 14), ('26', 14), ('19', 13), ('17', 13), ('40', 13), ('32', 12), ('24', 11), ('25', 11), ('15', 10), ('27', 9), ('41', 9), ('33', 9), ('18', 9), ('31', 8), ('23', 8), ('42', 8), ('16', 7), ('34', 7), ('14', 6), ('47', 6), ('36', 6), ('48', 5), ('43', 5), ('11', 5), ('13', 4), ('39', 4), ('9', 4), ('51', 4), ('38', 3), ('46', 3), ('50', 3), ('49', 2), ('83', 2), ('53', 2), ('61', 2), ('66', 2), ('52', 2), ('12', 2), ('63', 1), ('10', 1), ('56', 1), ('65', 1), ('78', 1), ('59', 1), ('6', 1), ('60', 1), ('62', 1), ('74', 1), ('58', 1)]
(len, count) (sorted by len):  [('83', 2), ('78', 1), ('74', 1), ('66', 2), ('65', 1), ('63', 1), ('62', 1), ('61', 2), ('60', 1), ('59', 1), ('58', 1), ('56', 1), ('53', 2), ('52', 2), ('51', 4), ('50', 3), ('49', 2), ('48', 5), ('47', 6), ('46', 3), ('43', 5), ('42', 8), ('41', 9), ('40', 13), ('39', 

Dataset:  tsar-es
Max length:  96
(len, count) (sorted by count): [('28', 25), ('38', 16), ('21', 15), ('42', 14), ('25', 13), ('16', 12), ('33', 11), ('37', 11), ('18', 11), ('31', 10), ('20', 10), ('35', 10), ('32', 9), ('23', 9), ('51', 8), ('30', 8), ('22', 8), ('47', 8), ('24', 7), ('15', 7), ('40', 7), ('34', 7), ('19', 7), ('26', 7), ('27', 7), ('54', 6), ('11', 6), ('9', 6), ('46', 5), ('29', 5), ('65', 5), ('43', 4), ('13', 4), ('44', 4), ('17', 4), ('48', 4), ('39', 4), ('66', 4), ('12', 4), ('64', 3), ('14', 3), ('49', 3), ('52', 3), ('45', 3), ('41', 3), ('115', 3), ('57', 3), ('80', 3), ('73', 2), ('70', 2), ('62', 2), ('106', 2), ('68', 2), ('7', 2), ('56', 2), ('36', 2), ('93', 1), ('61', 1), ('96', 1), ('78', 1), ('63', 1), ('75', 1), ('8', 1), ('72', 1), ('5', 1), ('50', 1), ('53', 1), ('138', 1), ('71', 1), ('10', 1), ('91', 1), ('100', 1)]
(len, count) (sorted by len):  [('138', 1), ('115', 3), ('106', 2), ('100', 1), ('96', 1), ('93', 1), ('91', 1), ('80', 3), ('78'

Dataset:  tsar-pt
Max length:  9
(len, count) (sorted by count): [('14', 19), ('19', 16), ('22', 16), ('25', 16), ('17', 16), ('15', 15), ('26', 14), ('21', 14), ('28', 13), ('27', 13), ('13', 13), ('18', 12), ('12', 11), ('11', 11), ('20', 10), ('16', 10), ('24', 10), ('34', 10), ('8', 10), ('32', 10), ('29', 9), ('10', 9), ('30', 9), ('33', 9), ('35', 8), ('31', 8), ('23', 8), ('36', 8), ('6', 7), ('37', 7), ('38', 5), ('39', 4), ('4', 4), ('40', 3), ('53', 3), ('5', 2), ('50', 2), ('49', 2), ('7', 2), ('3', 2), ('9', 2), ('42', 2), ('46', 2), ('55', 2), ('47', 2), ('41', 2), ('43', 2), ('44', 1), ('57', 1)]
(len, count) (sorted by len):  [('57', 1), ('55', 2), ('53', 3), ('50', 2), ('49', 2), ('47', 2), ('46', 2), ('44', 1), ('43', 2), ('42', 2), ('41', 2), ('40', 3), ('39', 4), ('38', 5), ('37', 7), ('36', 8), ('35', 8), ('34', 10), ('33', 9), ('32', 10), ('31', 8), ('30', 9), ('29', 9), ('28', 13), ('27', 13), ('26', 14), ('25', 16), ('24', 10), ('23', 8), ('22', 16), ('21', 14), 

# Analyse sentence split

In [6]:
import spacy


/home/kim/.pyenv/versions/3.7.5/envs/ConLS/lib/python3.7/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [16]:
#en 
def count_sentence_split(model, data):
    nlp = spacy.load(model)
    count = 0
    for row in tqdm(data.iloc, total=len(data)):
        doc = nlp(row['text'])
        sents = list(doc.sents)
        if len(sents) > 1:
            print(sents)
            count += 1
    return count
print('en:', count_sentence_split('en_core_web_sm', data_tsar_en))
print('es:', count_sentence_split('es_core_news_sm', data_tsar_es))
print('pt:', count_sentence_split('pt_core_news_sm', data_tsar_pt))


 81%|████████  | 313/386 [00:03<00:00, 107.33it/s]

[companies must raise £28 trillion chinese sovereign wealth fund stops buying euro debt surprise weakening in china import and exports, spain nationalises bankia as euro crisis escalates germany warns greece 'no bailout agreement, no aid' bankia, in which the state is taking a 45pc stake as a crisis measure to save    it from crippling bad loans, had €37.5bn in exposure to the property sector    at the end of 2011.]


100%|██████████| 386/386 [00:03<00:00, 105.02it/s]


en: 1


  3%|▎         | 10/381 [00:00<00:03, 97.43it/s]

...

 97%|█████████▋| 375/386 [00:02<00:00, 134.14it/s]

[para ele em condições ideais, o método funcionaria melhor em pacientes que tivessem um quadro estável com baixa contagem de vírus e forte presença de células de defesa]
[para uma dupla de paleoantropólogos dos estados unidos, o elemento que deu a partida na evolução da linhagem humana se encontra alguns palmos debaixo da terra e ainda faz qualquer um salivar diante de um prato de batata ou mandioca frita]


100%|██████████| 386/386 [00:02<00:00, 135.42it/s]

pt: 25


: 